# Ensemble by ranked list merging

In [1]:
from src.CBF.CBF_tfidf import ContentBasedFiltering
from src.utils.loader import *
from src.utils.evaluator import Evaluator

dataset = Dataset(load_tags=True,
                      filter_tag=False,
                      weight_tag=False)
print("Dataset loaded...")

Dataset loaded...


In [ ]:
from src.CBF.CBF_tfidf import ContentBasedFiltering
from src.UBF.UBF_CBF import UBF

